In [68]:
from fmp_python.fmp import FMP
import os
import pandas as pd
from SP500_div_yield_crawler import SP500
from datetime import datetime

os.environ["FMP_API_KEY"] = "606d643d87241cde956b5cd85a3c56d1"
fmp = FMP()

# dividend, date  - historical dividend values and dates
dividends = fmp.get_dividends('O', 'historical-price-full/stock_dividend')

# set proper DateTime object as the index of the dataframe
dividends['Datetime'] = pd.to_datetime(dividends['date'])
dividends = dividends.set_index('Datetime')

In [69]:


# resample annually and sum dividend values. Note that year end (Dec 31) will be shown for all groups
# dividends = dividends.resample("A")["adjDividend"].sum()
# dividends.dtypes

dividends_processed = dividends.resample("A")["date"].count()
today = datetime.today()
this_year = today.year
mask = (dividends_processed.index > (str(this_year-1)+'-01-01')) & (dividends_processed.index <= (str(this_year-1)+'-12-31'))
filtered_dividends=dividends_processed.loc[mask]
# print(filtered_df)
# include = dividends[dividends['Date'].datetime.year == this_year]

In [70]:

year = 2021
mask = (dividends.index > (str(year)+'-01-01')) & (dividends.index <= (str(year)+'-12-31'))
dividends_filtered_of_year=dividends.loc[mask]

list_of_div_months = dividends_filtered_of_year.index.tolist()
list_of_div_months = [date.strftime("%m") for date in list_of_div_months]
print (list_of_div_months)


['01']


In [71]:
short = fmp.get_quote_short('O')


In [72]:
# get dividend growth per year
dividends_new = pd.DataFrame()
# remove current year line as it cannot be complete and we are interested in historical data

dividends_new['yearlyDividendValue'] = dividends.resample("A")["adjDividend"].sum()
dividends_new['dividendGrowth'] = dividends_new.pct_change()
dividends_new = dividends_new.sort_index(ascending=False)

# mask out row of current year
mask = (dividends_new.index <= (str(this_year-1)+'-12-31'))
dividends_new_filtered=dividends_new.loc[mask]

print (dividends_new_filtered)

            yearlyDividendValue  dividendGrowth
Datetime                                       
2020-12-31             2.804000        0.031261
2019-12-31             2.719000        0.029924
2018-12-31             2.640000        0.040394
2017-12-31             2.537500        0.055532
2016-12-31             2.404000        0.054386
2015-12-31             2.280000        0.039840
2014-12-31             2.192646       -0.071020
2013-12-31             2.360272        0.327487
2012-12-31             1.778000        0.022611
2011-12-31             1.738686        0.008922
2010-12-31             1.723310        0.008705
2009-12-31             1.708438        0.024935
2008-12-31             1.666875        0.061112
2007-12-31             1.570875        0.084858
2006-12-31             1.448000        0.071085
2005-12-31             1.351900       -0.437505
2004-12-31             2.403400        0.219195
2003-12-31             1.971300        0.078274
2002-12-31             1.828200       -0